<a href="https://colab.research.google.com/github/ankesh86/RecommendationSystems/blob/main/Content_basedRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances
#from sklearn.feature_extraction import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import re
from gensim import models
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style
%matplotlib inline
from gensim.models import FastText as ft
from IPython.display import Image
import os

In [2]:
import gdown

word2vec_url = 'https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM'  # Replace 'YOUR_ACTUAL_ID' with the actual ID
word2vec_output = 'word2vec.bin.gz'
gdown.download(word2vec_url, word2vec_output, quiet=False)

!gunzip word2vec_output


Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=ace70729-2698-4348-b625-6b730d9b262b
To: /content/word2vec.bin.gz
100%|██████████| 1.65G/1.65G [00:47<00:00, 34.4MB/s]

gzip: word2vec_output.gz: No such file or directory


In [3]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2024-06-02 21:50:07--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-02 21:50:07--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.13MB/s    in 2m 40s  

2024-06-02 21:52:49 (5.13 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

In [12]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
#!gunzip cc.en.300.bin.gz

--2024-06-02 21:55:25--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.15, 108.157.254.121, 108.157.254.124, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz.1’

cc.en.300.bin.gz.1  100%[===================>]   4.19G  37.0MB/s    in 2m 25s  

2024-06-02 21:57:50 (29.5 MB/s) - ‘cc.en.300.bin.gz.1’ saved [4503593528/4503593528]



#Importing data as a Dataframe

In [17]:
Content_df = pd.read_csv("sample_data/Rec_sys_content.csv")
# Viewing Top 5 Rows
Content_df.head(5)

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


In [18]:
Content_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958 entries, 0 to 3957
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   StockCode     3958 non-null   object 
 1   Product Name  3958 non-null   object 
 2   Description   3958 non-null   object 
 3   Category      3856 non-null   object 
 4   Brand         3818 non-null   object 
 5   Unit Price    3943 non-null   float64
dtypes: float64(1), object(5)
memory usage: 185.7+ KB


In [19]:
Content_df.shape

(3958, 6)

In [20]:
Content_df.isnull().sum(axis=0)

StockCode         0
Product Name      0
Description       0
Category        102
Brand           140
Unit Price       15
dtype: int64

In [21]:
#importing Word2Vec
word2vecModel = models.KeyedVectors.load_word2vec_format('word2vec.bin.gz', binary=True)

In [22]:
#importing FastText
#fasttext_model = ft.load_fasttext_format("cc.en.300.bin")

In [23]:
#Import Glove
glove_df = pd.read_csv('glove.6B.300d.txt',sep=" ",
                       quoting=3, header=None, index_col=0)
glove_model = {key:value.values for key, value in glove_df.T.items()}

## Text pre-processing

In [24]:
## combining product and description
Content_df['Description'] = Content_df['Product Name'] + ' ' +Content_df['Description']

##dropping duplicates and keeping first records
unique_df = Content_df.drop_duplicates(subset=['Description'], keep='first')

## Converting String to Lower Case
unique_df['desc_lowered'] = unique_df['Description'].apply(lambda x:x.lower())

## remove Stop special characters
unique_df['desc_lowered'] = unique_df['desc_lowered'].apply(lambda x:re.sub(r'[^\w\s]','',x))

#converting description to list
desc_list = list(unique_df['desc_lowered'])
unique_df = unique_df.reset_index(drop=True)

<ipython-input-24-b2228480fb5b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['desc_lowered'] = unique_df['Description'].apply(lambda x:x.lower())
<ipython-input-24-b2228480fb5b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['desc_lowered'] = unique_df['desc_lowered'].apply(lambda x:re.sub(r'[^\w\s]','',x))


# **Text to Features**



1.   One Hot encoding
2.   Count Vectorizer
3.   TF-IDF



## **Word Embedding**

In [26]:
#importing count vectorizer
cnt_vec = CountVectorizer(stop_words = 'english')

In [27]:
#importing IFIDF
tfidf_vec = TfidfVectorizer(stop_words='english', analyzer='word', ngram_range=(1,3))

## **Similarity measures**

1. Eucleadian distance
2. Cosine similarity
3. Manhattan distance

In [28]:
#Eculedian distance
def find_euclidean_distances(sim_matrix, index, n=10):
  #Getting Score and Index
  result = list(enumerate(sim_matrix[index]))

  #Sorting the Score and taking top 10 products
  sorted_result = sorted(result, key=lambda x:x[1], reverse=False)[1:10+1]

  #Mapping index with data
  similar_products = [{'value': unique_df.iloc[x[0]]['Product Name'], 'score':round(x[1], 2)} for x in sorted_result]

  return similar_products

In [42]:
def find_similarity(cosine_sim_matrix, index, n=10):

    # calculate cosine similarity between each vectors
    result = list(enumerate(cosine_sim_matrix[index]))

    # Sorting the Score
    sorted_result = sorted(result,key=lambda x:x[1],reverse=True)[1:n+1]

    similar_products =  [{'value': unique_df.iloc[x[0]]['Product Name'], 'score' : round(x[1], 2)} for x in sorted_result]

    return similar_products

In [30]:
#Manhattan similarity
def find_manhattan_distance(sim_matrix, index, n=10):
  #Getting Score and Index
  result = list(enumerate(sim_matrix[index]))

  #Sorting the Score and taking top 10 products
  sorted_result = sorted(result, key=lambda x:x[1], reverse=False)[1:10+1]

  #Mapping index with data
  similar_products = [{'value': unique_df.iloc[x[0]]['Product Name'], 'score':round(x[1], 2)} for x in sorted_result]

  return similar_products

## **Build a model using CountVectorizer**

In [45]:
from math import e
from operator import index
#comparing similarity to ge tthe top matches using count Vec
def get_recommendation_cv(product_id, df, similarity, n=10):

  row = df.loc[df['Product Name'] == product_id]
  index = list(row.index)[0]
  description = row['desc_lowered'].loc[index]

  #create vector using Count Vectorizer
  count_vector = cnt_vec.fit_transform(desc_list)

  if similarity == "cosine":
    sim_matrix = cosine_similarity(count_vector)
    products = find_similarity(sim_matrix , index)
  elif similarity == "manhattan":
    sim_matrix = manhattan_distances(count_vector)
    products = find_manhattan_distance(sim_matrix, index)
  else:
    sim_matrix = euclidean_distances(count_vector)
    products = find_euclidean_distances(sim_matrix, index)

  return products

In [46]:
product_id = 'Vickerman 14" Finial Drop Christmas Ornaments, Pack of 2'

# Cosine Similarity
get_recommendation_cv(product_id, unique_df, similarity = "cosine", n=10)

[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.28},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 0.2},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 0.19},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 0.19},
 {'value': 'PeanutsÃ‚Â Valentine Sign Craft Kit (Pack of 12)', 'score': 0.13},
 {'value': 'Simplicity 3 Pack Elastic Hair Ties White/Pink/Pink Leopard, 24 Count',
  'score': 0.13},
 {'value': '3 Pack Newbee Fashion- "Butterfly" Thin Design Gold Emblem Reading Glasses with Lanyard +1.75',
  'score': 0.12},
 {'value': 'Paper Mate Write Bros. Grip Mechanical Pencil, 0.7mm 5 ea (Pack of 2)',
  'score': 0.11},
 {'value': 'Christopher Radko Glass Plum Frosty S

In [47]:
# Manhattan Similarity
get_recommendation_cv(product_id, unique_df, similarity = "manhattan", n=10)

[{'value': 'Stepping Stones', 'score': 43.0},
 {'value': 'Global Portuguese', 'score': 43.0},
 {'value': 'Polo Blue by Ralph Lauren', 'score': 43.0},
 {'value': 'Auburn Leathercrafters Tuscany Leather Dog Collar', 'score': 45.0},
 {'value': 'Leftover Salmon', 'score': 45.0},
 {'value': 'Good (Vinyl)', 'score': 45.0},
 {'value': 'Drunken Monkeys', 'score': 45.0},
 {'value': 'DuraTech Roof Support Trim', 'score': 47.0},
 {'value': 'Amerlite Niche Sealing Ring', 'score': 47.0},
 {'value': 'Learning and Performance in Corrections', 'score': 47.0}]

In [48]:
# Euclidean Similarity
get_recommendation_cv(product_id, unique_df, similarity = "euclidean", n=10)

[{'value': 'Polo Blue by Ralph Lauren', 'score': 9.0},
 {'value': 'Auburn Leathercrafters Tuscany Leather Dog Collar', 'score': 9.11},
 {'value': 'Global Portuguese', 'score': 9.11},
 {'value': 'Stepping Stones', 'score': 9.22},
 {'value': 'Always in My Heart', 'score': 9.22},
 {'value': 'Leftover Salmon', 'score': 9.22},
 {'value': 'Good (Vinyl)', 'score': 9.22},
 {'value': 'Drunken Monkeys', 'score': 9.22},
 {'value': 'Learning and Performance in Corrections', 'score': 9.43},
 {'value': 'Chasing Hamburg (Vinyl)', 'score': 9.43}]

## **Build a Model using TF-IDF features**

In [49]:
from math import e
from operator import index
#comparing similarity to ge tthe top matches using count Vec
def get_recommendation_tfidf(product_id, df, similarity, n=10):

  row = df.loc[df['Product Name'] == product_id]
  index = list(row.index)[0]
  description = row['desc_lowered'].loc[index]

  #create vector using Count Vectorizer
  tfidf_matrix = tfidf_vec.fit_transform(desc_list)

  if similarity == "cosine":
    sim_matrix = cosine_similarity(tfidf_matrix)
    products = find_similarity(sim_matrix , index)
  elif similarity == "manhattan":
    sim_matrix = manhattan_distances(tfidf_matrix)
    products = find_manhattan_distance(sim_matrix, index)
  else:
    sim_matrix = euclidean_distances(tfidf_matrix)
    products = find_euclidean_distances(sim_matrix, index)

  return products

In [50]:
# Cosine Similarity
get_recommendation_tfidf(product_id, unique_df, similarity = "cosine", n=10)

[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.07},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 0.05},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 0.05},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 0.05},
 {'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 0.02},
 {'value': 'Christopher Radko Glass Plum Frosty Snowman Christmas Ornament #1017624',
  'score': 0.02},
 {'value': 'CMFUN Watercolor Brush Creative Flower Made with Ink Hand Painting for Your Designs Pillowcase 20x20 inch',
  'score': 0.02},
 {'value': 'SKIN DECAL FOR OtterBox Symmetry Samsung Galaxy S7 Case - Christmas Snowflake Blue Ornaments DECAL, NOT A CAS

In [51]:
# Manhattan Similarity
get_recommendation_tfidf(product_id, unique_df, similarity = "manhattan", n=10)

[{'value': 'Stepping Stones', 'score': 11.05},
 {'value': 'Global Portuguese', 'score': 11.42},
 {'value': 'Polo Blue by Ralph Lauren', 'score': 11.68},
 {'value': 'DuraTech Roof Support Trim', 'score': 12.06},
 {'value': 'Silver Metallic Foil Streamers', 'score': 12.07},
 {'value': 'Pro Pinstripe Jersey Sleeveles', 'score': 12.07},
 {'value': 'Amerlite Niche Sealing Ring', 'score': 12.07},
 {'value': 'Leftover Salmon', 'score': 12.3},
 {'value': 'Drunken Monkeys', 'score': 12.32},
 {'value': "Alfred's Drum Method, Book 1", 'score': 12.41}]

In [52]:
# Euclidean Similarity
get_recommendation_tfidf(product_id, unique_df, similarity = "euclidean", n=10)

[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 1.37},
 {'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 1.38},
 {'value': '12ct Light Gunmetal Gray Shatterproof 4-Finish Christmas Ball Ornaments 4" (100mm)',
  'score': 1.38},
 {'value': '16 inches 40 inches "MERRY CHRISTMAS" Letter Foil Inflated Balloon Float Helium Aluminum Balloons for Christmas Decoration',
  'score': 1.38},
 {'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 1.4},
 {'value': 'Christopher Radko Glass Plum Frosty Snowman Christmas Ornament #1017624',
  'score': 1.4},
 {'value': 'CMFUN Watercolor Brush Creative Flower Made with Ink Hand Painting for Your Designs Pillowcase 20x20 inch',
  'score': 1.4},
 {'value': 'SKIN DECAL FOR OtterBox Symmetry Samsung Galaxy S7 Case - Christmas Snowflake Blue Ornaments DECAL, NOT A CASE',

## **Build a model using Word2vec Features**

In [56]:
#Comparing similarity to get the top matches using Word2vec pretrained model
def get_recommendation_word2vec(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lowered'].loc[input_index]

    #create vectors for each desc using word2vec
    vector_matrix = np.empty((len(desc_list), 300))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((300,))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += word2vecModel[each_word]
                count += 1
            except:
                continue

        vector_matrix[index] = sentence_vector

    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , input_index)

    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , input_index)

    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , input_index)

    return products

In [57]:
# Cosine Similarity
get_recommendation_word2vec(product_id, unique_df, similarity = "cosine", n=10)

[{'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.85},
 {'value': 'Red Gift Wrap Bag by K-Kraft (Festive Red Stripes, 13 x 17.5 inches)',
  'score': 0.85},
 {'value': '12 Pcs 3D Butterfly Stickers DIY Mural Art Decal Wall Stickers Crafts Wall Paper Decor (Yellow Series)',
  'score': 0.84},
 {'value': 'Cat Mod Garden Complex - Unfinished/Black', 'score': 0.83},
 {'value': 'Three Things That Makes Good Coffee: Sugar, Sugar and More Sugar Wall Decal - wall decal, sticker, quotes and sayings - W5082 - Light pink, 35in x 18in',
  'score': 0.82},
 {'value': 'Halloween Bats Wall Decal - Wall Sticker, Vinyl Wall Art, Home Decor, Wall Mural - W5014 - Royal blue, 16in x 10in',
  'score': 0.82},
 {'value': 'Hibiscus Flowers Wall Decal - Wall Sticker, Vinyl Wall Art, Home Decor, Wall Mural - 2186 - 8in x 19in, Dark green',
  'score': 0.82},
 {'value': 'Abstract Heart Plant Wall Decal - Wall Sticker, Vi

In [58]:
# Manhattan Similarity
get_recommendation_word2vec(product_id, unique_df, similarity = "manhattan", n=10)

[{'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 458.13},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 488.19},
 {'value': 'Cavalier Spaniel St. Patricks Day Shamrock Mouse Pad&#44; Hot Pad Or Trivet',
  'score': 497.0},
 {'value': "Call of the Wild Howling the Full Moon Women's Racerback Alpha Wolf",
  'score': 509.22},
 {'value': 'Fringe Table Skirt Purple 9 ft x 29 inches Pkg/1',
  'score': 516.08},
 {'value': 'Trend Enterprises T-83315 1.25 in. Holiday Pals & Peppermint Scratch N Sniff Stinky Stickers&#44; Large Round',
  'score': 522.0},
 {'value': "Allwitty 1039 - Women's T-Shirt Ipac Pistol Gun Apple Iphone Parody",
  'score': 525.03},
 {'value': 'Clear 18 Note Acrylic Box Musical Paperweight - Light My Fire',
  'score': 526.08},
 {'value': 'Handcrafted Ercolano Music Box Featuring "Luncheon of the Boating Party" by Renoir, Pierre Auguste - New YorkNew York',
  'score': 527.88},
 {'va

In [59]:
# Euclidean Similarity
get_recommendation_word2vec(product_id, unique_df, similarity = "euclidean", n=10)

[{'value': 'storefront christmas LED Decoration Light Gold Color Star Shape Vine Wedding Party event',
  'score': 32.76},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 34.82},
 {'value': 'Cavalier Spaniel St. Patricks Day Shamrock Mouse Pad&#44; Hot Pad Or Trivet',
  'score': 35.78},
 {'value': "Call of the Wild Howling the Full Moon Women's Racerback Alpha Wolf",
  'score': 36.09},
 {'value': 'Fringe Table Skirt Purple 9 ft x 29 inches Pkg/1', 'score': 36.28},
 {'value': "New Way 075 - Men's Sleeveless Fbi Female Body Inspector",
  'score': 37.43},
 {'value': 'Trend Enterprises T-83315 1.25 in. Holiday Pals & Peppermint Scratch N Sniff Stinky Stickers&#44; Large Round',
  'score': 37.51},
 {'value': 'Platinum 5 mm Comfort Fit Half Round Wedding Band - Size 9.5',
  'score': 37.6},
 {'value': 'Christmas Presents Between Friends Chinese Crested Black Sticky Note Holder',
  'score': 37.78},
 {'value': 'Tulane Green Wave Muscle Mug', 'score': 37.85}]

## **fastText**
fastText is another word embedding method that is an extension of the word2vec model. Instead of learning vectors for words directly, fastText represents each word as an n-gram of characters. So, for example, take the word, “artificial” with n=3, the fastText representation of this word is <ar, art, rti, tif, ifi, fic, ici, ial, al>, where the angular brackets indicate the beginning and end of the word. This helps capture the meaning of shorter words and allows the embeddings to understand suffixes and prefixes.

In [ ]:
# Comparing similarity to get the top matches using fastText pretrained model

def get_recommendation_fasttext(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lowered'].loc[input_index]

    #create vectors for each description using fasttext
    vector_matrix = np.empty((len(desc_list), 300))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((300,))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += fasttext_model.wv[each_word]
                count += 1
            except:
                continue

        vector_matrix[index] = sentence_vector

    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , input_index)

    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , input_index)

    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , input_index)

    return products
# Cosine Similarity
get_recommendation_fasttext(product_id, unique_df, similarity = "cosine", n=10)


## **Build Model using GloVe Features**

The advantage of GloVe is that, unlike Word2vec, GloVe does not rely just on local statistics (local context information of words), but incorporates global statistics (word co-occurrence) to obtain word vectors. But keep in mind that there’s quite a bit of synergy between the GloVe and Word2vec.

In [62]:
# Comparing similarity to get the top matches using Glove pretrained model

def get_recommendation_glove(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lowered'].loc[input_index]

    #using glove embeddings to create vectors
    vector_matrix = np.empty((len(desc_list), 300))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((300,))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += glove_model[each_word]
                count += 1

            except:
                continue

        vector_matrix[index] = sentence_vector


    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , input_index)

    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , input_index)

    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , input_index)

    return products

In [63]:
# Cosine Similarity
get_recommendation_glove(product_id, unique_df, similarity = "cosine", n=10)

[{'value': 'Red Gift Wrap Bag by K-Kraft (Festive Red Stripes, 13 x 17.5 inches)',
  'score': 0.94},
 {'value': 'Cat Mod Garden Complex - Unfinished/Black', 'score': 0.93},
 {'value': 'CafePress - FIN Nuts Squirrel - Cute Infant Baby Football Bodysuit',
  'score': 0.93},
 {'value': 'American Foxhound Christmas Sticky Note Holder BB8433SN',
  'score': 0.93},
 {'value': 'CHAIM Street Sign Childrens Name Room Sign | Indoor/Outdoor | \xa024" Wide',
  'score': 0.93},
 {'value': '12 Pcs 3D Butterfly Stickers DIY Mural Art Decal Wall Stickers Crafts Wall Paper Decor (Yellow Series)',
  'score': 0.93},
 {'value': 'All Weather Cornhole Bags - Set of 8', 'score': 0.93},
 {'value': 'Ecclesiastes 8:15 so I commend the enjoymentâ€¦ Vinyl Decal Sticker Quote - Small - Metallic Gold',
  'score': 0.93},
 {'value': 'Fancyleo Christmas Glasses Frames 2 Pack Glittered Eyeglasses Glasses Set No Lens Kids Family Xmas Party Ornaments Gift',
  'score': 0.93},
 {'value': 'Buckle-Down DL-W30350-N Narrow 0.5" C

In [64]:
# Euclidean Similarity
get_recommendation_glove(product_id, unique_df, similarity = "euclidean", n=10)

[{'value': 'Is It To Late To Be Good Grinch Christmas Mens Tank Top Shirt',
  'score': 87.3},
 {'value': "New Way 075 - Men's Sleeveless Fbi Female Body Inspector",
  'score': 88.33},
 {'value': '8 1/2 x 14 Cardstock - Crystal Metallic (500 Qty.)',
  'score': 89.96},
 {'value': "Allwitty 1039 - Women's T-Shirt Ipac Pistol Gun Apple Iphone Parody",
  'score': 90.29},
 {'value': '031 - Unisex Long-Sleeve T-Shirt Disobey V For Vendetta Anonymous Fawkes Mask',
  'score': 91.6},
 {'value': "1042 - Unisex Long-Sleeve T-Shirt Free Hugs And Maybe A Kiss If You're Lucky",
  'score': 92.37},
 {'value': 'Skin Decal For Beats By Dr. Dre Beats Pill Plus / Sugar Skull Girl',
  'score': 92.46},
 {'value': 'Spiral Birthday Candles, 36 Count', 'score': 94.19},
 {'value': 'IN-70/65 Blue Paper Streamers 2PK', 'score': 95.41},
 {'value': '(2-Pack) StealthShields Tablet Screen Protector for Lenovo IdeaPad Yoga 11 (U...',
  'score': 95.92}]

## **co-occurrence matrix**
The purpose of this matrix is to present the number of times each word appears in the same context. .

In [67]:
# create cooccurence matrix

#preprocessing
df = Content_df.head(250)
# Combining Product and Description
df['Description'] = df['Product Name'] + ' ' +df['Description']
unique_df = df.drop_duplicates(subset=['Description'], keep='first')
unique_df['desc_lowered'] = unique_df['Description'].apply(lambda x: x.lower())
unique_df['desc_lowered'] = unique_df['desc_lowered'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
desc_list = list(unique_df['desc_lowered'])

co_ocr_vocab = []
for i in desc_list:
    [co_ocr_vocab.append(x) for x in i.split()]

co_occur_vector_matrix = np.zeros((len(co_ocr_vocab), len(co_ocr_vocab)))

for _, sent in enumerate(desc_list):
    words = sent.split()
    for index, word in enumerate(words):
        if index != len(words)-1:
            co_occur_vector_matrix[co_ocr_vocab.index(word)][co_ocr_vocab.index(words[index+1])] += 1

<ipython-input-67-df54c66624a9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'] = df['Product Name'] + ' ' +df['Description']
<ipython-input-67-df54c66624a9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_df['desc_lowered'] = unique_df['Description'].apply(lambda x: x.lower())
<ipython-input-67-df54c66624a9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [68]:
# Comparing similarity to get the top matches using cooccurence matrix

def get_recommendation_coccur(product_id, df, similarity, n=10):

    row = df.loc[df['Product Name'] == product_id]
    input_index = list(row.index)[0]
    description = row['desc_lowered'].loc[input_index]

    vector_matrix = np.empty((len(desc_list), len(co_ocr_vocab)))
    for index, each_sentence in enumerate(desc_list):
        sentence_vector = np.zeros((len(co_ocr_vocab),))
        count  = 0
        for each_word in each_sentence.split():
            try:
                sentence_vector += co_occur_vector_matrix[co_ocr_vocab.index(each_word)]
                count += 1

            except:
                continue

        vector_matrix[index] = sentence_vector/count


    if similarity == "cosine":
        sim_matrix = cosine_similarity(vector_matrix)
        products = find_similarity(sim_matrix , index)

    elif similarity == "manhattan":
        sim_matrix = manhattan_distances(vector_matrix)
        products = find_manhattan_distance(sim_matrix , index)

    else:
        sim_matrix = euclidean_distances(vector_matrix)
        products = find_euclidean_distances(sim_matrix , index)

    return products

In [69]:
# Cosine Similarity
get_recommendation_coccur(product_id, unique_df, similarity = "cosine", n=10)

[{'value': 'Medi Comfort Closed Toe Knee Highs -15-20 mmHg Reg',
  'score': 0.94},
 {'value': "JustVH Women's Solid Henley V-Neck Casual Blouse Pleated Button Tunic Shirt Top",
  'score': 0.94},
 {'value': 'Toddler Kid Boys Girls Lightweight Breathable Trendy Slip-on Sneaker (6M US Toddler, Red)',
  'score': 0.94},
 {'value': "Pull-Ups Girls' Learning Designs Training Pants (Choose Pant Size and Count)",
  'score': 0.94},
 {'value': 'Ebe Reading Glasses Mens Womens Amber Red Oval Round Full Frame Anti Glare grade ckbdp9118',
  'score': 0.92},
 {'value': "CafePress - Surf's Up! - Men's Light Pajamas", 'score': 0.91},
 {'value': 'Sculpey Polymer Clay Just Orange', 'score': 0.91},
 {'value': 'Ebe Women Reading Glasses Reader Cheaters Anti Reflective Lenses TR90 ry2209',
  'score': 0.9},
 {'value': "Dr. Comfort Paradise Women's Casual Shoe: 4.5 X-Wide (E-2E) Black Velcro",
  'score': 0.9},
 {'value': 'Basics - Blank Adult Hoodie Sweatshirt', 'score': 0.89}]